<a href="https://colab.research.google.com/github/smwongela/NLP_Transformers/blob/main/accelerate_mrpc_classification_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install transformers
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)


In [47]:
! pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-8shcai4b
  Running command git clone -q https://github.com/huggingface/accelerate /tmp/pip-req-build-8shcai4b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [48]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 53.1 MB/s 
     |████████████████████████████████| 145 kB 58.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=68f5c1807585b70feb8074d6d66046d4c8370c0039248d1d5fa8f37cb3310eca
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [50]:
import torch
from torch.utils.data import DataLoader
from accelerate import Accelerator, DistributedType

In [51]:
import wandb
# Import wandb
## wandb: You can find your API key in your browser here: https://wandb.ai/authorize

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [52]:
from datasets import load_dataset,load_metric

In [53]:
from transformers import(
     AutoTokenizer, 
     DataCollatorWithPadding,
     AutoModelForSequenceClassification,
     get_linear_schedule_with_warmup,
     set_seed,
     )
from tqdm.auto import tqdm
import datasets
import transformers

In [54]:
##Load datasets
raw_datasets=load_dataset("glue","mrpc")

  0%|          | 0/3 [00:00<?, ?it/s]

In [55]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [56]:
raw_datasets['train'][0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [57]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(raw_datasets["train"])

,sentence1,sentence2,label,idx
0,"As of Tuesday , almost 250 health-care workers were in quarantine .","In addition , 6,800 people were in quarantine and thousands of health-care workers in working quarantine .",not_equivalent,632
1,""" This fire is going to have a great potential to get into those areas , "" Oltrogge said .",""" The fire is going to have great potential to get in there tomorrow , "" he said .",equivalent,2390
2,"However , prosecutors have declined to take criminal action against guards , though Fine said his inquiry is not finished .","Prosecutors have declined to take criminal action against corrections officers , although Fine said his inquiry was not finished .",equivalent,3285
3,"The University of Michigan released a new undergraduate admission process Thursday , dropping a point system the U.S. Supreme Court found unconstitutional in June .",The University of Michigan released today a new admissions policy after the U.S. Supreme Court struck down in June the way it previously admitted undergraduates .,equivalent,68
4,"But he added : "" You can 't win elections by looking in the rearview mirror . """,""" You can 't win elections by looking through the rear view mirror , "" he said .",equivalent,1937
5,""" I think 70 percent of the work is already done , "" Tauzin told reporters in the Capitol .",""" About 70 percent of the work is already done , "" Mr. Tauzin said .",equivalent,451
6,""" Arifin has disclosed to the I.S.D. that he is involved with a group of like-minded individuals in planning terrorist attacks against certain targets in Thailand , "" the statement said .",""" Arifin has disclosed . . . that he is involved with a group of like-minded individuals in planning terrorist attacks against certain targets in Thailand .",equivalent,2054
7,"Dr Blix said : "" I don 't know exactly how they calculated this figure .","Mr Blix said : "" I don 't know exactly how they calculated this figure of 45 minutes in the dossier .",equivalent,3318
8,Staff writer Dave Michaels contributed to this report .,Staff writers Frank Trejo and Robert Ingrassia contributed to this report .,not_equivalent,3724
9,"Shares of USA Interactive rose $ 2.28 , or 7 percent , to $ 34.96 on Friday in Nasdaq Stock Market composite trading and have gained 53 percent this year .","Shares of LendingTree rose $ 6.03 , or 41 percent , to close at $ 20.72 on the Nasdaq stock market yesterday .",not_equivalent,625


In [58]:
#Define model check point and tokenizer
checkpoint="bert-base-cased"
tokenizer= AutoTokenizer.from_pretrained(checkpoint)

In [59]:
tokenizer("Hello, first sentence", "Hello second sentence")

{'input_ids': [101, 8667, 117, 1148, 5650, 102, 8667, 1248, 5650, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [60]:

def tokenize_function(example):
  outputs = tokenizer(example['sentence1'], example['sentence2'],truncation=True, padding="max_length",max_length=128)
  return outputs

In [61]:
tokenize_function(raw_datasets['train'][:5])

{'input_ids': [[101, 7277, 2180, 5303, 4806, 1117, 1711, 117, 2292, 1119, 1270, 107, 1103, 7737, 107, 117, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 11336, 6732, 3384, 1106, 1140, 1112, 1178, 107, 1103, 7737, 107, 117, 7277, 2180, 5303, 4806, 1117, 1711, 1104, 9938, 4267, 12223, 21811, 1117, 2554, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 10684, 2599, 9717, 1161, 2205, 11288, 1377, 112, 188, 1196, 4147, 1103, 4129, 1106, 19770, 2787, 1107, 1772, 1111, 109, 123, 119, 126, 3775, 119, 102, 10684, 2599, 9717, 1161, 3306, 11288, 1377, 112, 188, 1107, 1876, 1111, 109, 5691, 1495, 1550, 1105, 1962, 1122, 1106, 19770, 2787, 1111, 109, 122, 119, 129, 3775, 1107, 1772, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [62]:
## Tokenize the dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True,remove_columns=["idx","sentence1","sentence2"] )

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [63]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [64]:
tokenized_datasets = tokenized_datasets.rename_column("label","labels")

In [65]:
tokenized_datasets["train"].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [66]:
tokenized_datasets.set_format("torch")

A look at the model using sequence classification head


In [67]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [68]:
##Define Dataloaders to convert data elements into batches and iterate over them

In [69]:
from torch.utils.data import DataLoader
def create_dataloaders(train_batch_size=8, eval_batch_size=32):
  train_dataloader = DataLoader(
      tokenized_datasets['train'],
      shuffle=True,
      batch_size=train_batch_size,
      #collate_fn= data_collator
  )
  eval_dataloader = DataLoader(
      tokenized_datasets['validation'],
      shuffle= False,
      batch_size= eval_batch_size,
      #collate_fn= data_collator
  ) 
  return train_dataloader, eval_dataloader
train_dataloader, eval_dataloader= create_dataloaders()
#Let's have a look at our train and evaluation dataloaders to check a batch can go through the model.
for batch in train_dataloader:
  break
{k: v.shape for k, v in batch.items()}


{'attention_mask': torch.Size([8, 128]),
 'input_ids': torch.Size([8, 128]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 128])}

In [70]:
outputs=model(**batch)
outputs

SequenceClassifierOutput([('loss', tensor(0.7254, grad_fn=<NllLossBackward>)),
                          ('logits', tensor([[-0.5152, -0.7210],
                                   [-0.5189, -0.7270],
                                   [-0.5221, -0.7191],
                                   [-0.5055, -0.7227],
                                   [-0.4989, -0.7493],
                                   [-0.5086, -0.7440],
                                   [-0.5254, -0.7133],
                                   [-0.5223, -0.7365]], grad_fn=<AddmmBackward>))])

et's have a look at our train and evaluation dataloaders to check a batch can go through the model.

In [71]:
##provide metric for model evaluation

metric= load_metric("glue","mrpc")

In [72]:
##use this object on some predictions by calling the compute function

In [73]:
logits= outputs.logits
print(logits)

tensor([[-0.5152, -0.7210],
        [-0.5189, -0.7270],
        [-0.5221, -0.7191],
        [-0.5055, -0.7227],
        [-0.4989, -0.7493],
        [-0.5086, -0.7440],
        [-0.5254, -0.7133],
        [-0.5223, -0.7365]], grad_fn=<AddmmBackward>)


In [74]:
predictions = torch.argmax(logits, dim=-1)

In [75]:
metric.compute(predictions=predictions, references=batch["labels"])

{'accuracy': 0.375, 'f1': 0.0}

# FINE TUNING 

In [76]:
###Fine Tuning our model
hyperparameters={
    "learning_rate":3e-5,
    "num_epochs":3,
    "train_batch_size": 8, #Actual batch size will be this x 8
    "eval_batch_size": 32,#Actual batch size will be this x 8
    "seed":42,
    
}

In [77]:
from transformers import AdamW

In [88]:
from transformers.utils.dummy_pt_objects import MODEL_FOR_SEMANTIC_SEGMENTATION_MAPPING
import accelerate
def training_function(model):
   # Initialize accelerator
   wandb.init()
   accelerator = Accelerator(log_with="wandb")
   accelerator.init_trackers("mrpc_classification")
   
   wandb.watch(model)
   # To have only one message (and not 8) per logs of Transformers or Datasets, we set the logging verbosity
   # to INFO for the main process only.
   if accelerator.is_main_process:
      datasets.utils.logging.set_verbosity_warning()
      transformers.utils.logging.set_verbosity_info()
   else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

   train_dataloader,eval_dataloader= create_dataloaders(
        train_batch_size=hyperparameters["train_batch_size"], eval_batch_size=hyperparameters["eval_batch_size"]
    )
    #set the seed befor model instantiation as it determines the random head
   set_seed(hyperparameters["seed"])

  
    #instantiate the optimizer
   optimizer = AdamW(params=model.parameters(), lr= hyperparameters["learning_rate"])
    #unpack the objects in the same order we gave them to the
    # prepare method.
   model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader,eval_dataloader)
   num_epochs=hyperparameters["num_epochs"]

   # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
   lr_scheduler= get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader)*num_epochs,

    )

    # Instantiate a progress bar to keep track of training. Note that we only enable it on the main
    # process to avoid having 8 progress bars.
   progress_bar = tqdm(range(num_epochs * len(train_dataloader)), disable=not acclerator.is_main_process)
    # Now we train the model
   for epoch in range(num_epochs):
     model.train()
     for step, batch in enumerate(train_dataloader):
        outputs=model(**batch)
        loss=outputs.loss
        wandb.log({"loss": loss}) # Log the Training Loss to W&B
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

     model.eval()
     all_predictions=[]
     all_labels=[]

     for step, batch in enumerate(eval_dataloader):
       with torch.no_grad():
          outputs= model(**batch)
       predictions = outputs.logits.argmax(dim=-1)

        # We gather predictions and labels from the 8 TPUs to have them all.

       all_predictions.append(accelerator.gather(predictions))
       all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.

       all_predictions = torch.cat(all_predictions)[:len(tokenized_datasets["validation"])]
       all_labels = torch.cat(all_labels)[:len(tokenized_datasets["validation"])]

       eval_metric = metric.compute(predictions= all_predictions, reference=all_labels)

       wandb.log({'Epoch': epoch, 'Eval_Metric':eval_metric}) # Log the Epoch and ROC AUC Score
        
        # Use accelerator.print to print only on the main process.


       accelerator.print(f"epoch {epoch}:", eval_metric)
    
        


In [ ]:
#Launch

from accelerate import notebook_launcher
notebook_launcher(training_function, (model,))

Launching a training on 8 TPU cores.
